## NIM Agent Blueprint for Generative Virtual Screening in Drug Discovery
This example notebook demonstrates how to connect BioNeMo NIMs to carry out a few key steps of a virtual screening workflow. Importantly, these steps are powered by highly performant AI models in each category: AlphaFold2 for folding, MolMIM for molecular generation, and DiffDock for protein-ligand docking.

Below, we illustrate this workflow using an example protein and example molecule of interest, the SARS-CoV-2 main protease and Nirmatrelvir, however, the user is free to define any protein and molecule of their choosing.

All of these capabilities are enabled by NVIDIA NIM and NVIDIA NIM Blueprints. For more details, please visit [NVIDIA NIM Blueprints](https://build.nvidia.com/nim/blueprints).

### BioNeMo Configurations
Before you begin, please set the NGC_CLI_API_KEY environment variable to a personal run key for your NGC Org and Team before running docker compose. Then, you can spin the NIMs up using the following docker command from the same directory as the `docker-compose.yaml`:

`docker compose up`

In [15]:
import requests

#AF2_HOST = 'http://localhost:8081'
DIFFDOCK_HOST = 'http://localhost:8082'
MOLMIM_HOST = 'http://localhost:8083'

In [29]:
import os
import shutil
import json
import pandas as pd

### Get folded protein 

In [17]:
import sys
sys.path.append("..")

from src import utils

In [18]:
print(utils)

<module 'src.utils' from '/home/substrate/projects/generative-virtual-screening/notebooks/../src/utils.py'>


In [19]:

# predicted target protein file path
protein_file_path = "../data/protein_input_files/mpro_sarscov2.pdb"

# TODO: to be replaced by utils
def file_to_json_compatible_string(file_path):
    """
    Convert PDB file and sdf file to JSON
    """
    with open(file_path, 'r') as file:
        content_str = file.read()
    return content_str

folded_protein = file_to_json_compatible_string(protein_file_path)
#folded_protein = utils.file_to_json_compatible_string(protein_file_path)

### Molecular Generation with MolMIM
The next step in our workflow is generating molecules with optimized chemical properties starting from a seed molecule of interest. Here, molecular generation is powered by MolMIM, an LLM-inspired model aimed at generating and optimizing molecules according to user-defined objectives. The "MIM" part of MolMIM stands for Mutual Information Machine, which describes the mutual-information-based loss used to preserve chemical similarity in the model's latent space.

Here, we begin with Nirmatrelvir, an active component of the Covid treatment Paxlovid, aimed at targeting the SARS-CoV-2 main protease. By using this molecule as the input to MolMIM, the model will return 5 generated molecules with the highest chemical similarity to MolMIM. The user is able to specify the number of generated molecules to return when querying the MolMIM NIM.

Additionally, the user is able to specify chemical properties to optimize for. In this example, we have chosen to optimize the Quantitative Estimate of Drug-Likeness (QED) score, to produce molecules with favorable pharmacokinetic properties.

Note especially that here we're using the `/generate` endpoint of the MolMIM NIM.  But MolMIM was designed for controlled generation with user-defined oracles.  For this type of application you will want to call the `/decode` endpoint.  See the [documentation](https://docs.nvidia.com/nim/bionemo/molmim/latest/overview.html#decode) and [example notebook](https://github.com/NVIDIA/BioNeMo/blob/main/examples/service/notebooks/cma_custom_oracles.ipynb) for additional information about using user-defined oracles.

In [37]:
# Nirmatrelvir
#molecule = "CC1(C2C1C(N(C2)C(=O)C(C(C)(C)C)NC(=O)C(F)(F)F)C(=O)NC(CC3CCNC3=O)C#N)C"
#molecule_name = "Nirmatrelvir"

df_starting_molecules = pd.read_csv('../data/starting_molecule_smiles.csv')
molecule_name = df_starting_molecules['Molecules'][1]
molecule = df_starting_molecules['Smiles'][1]

print(molecule_name)

Ensitrelvir


In [38]:
molmim_response = requests.post(
    f'{MOLMIM_HOST}/generate',
    json={
        'smi': molecule,
        'num_molecules': 5,
        'algorithm': 'CMA-ES',
        'property_name': 'QED',
        'min_similarity': 0.7, # Ignored if algorithm is not "CMA-ES".
        'iterations': 10,
    }).json()

In [39]:
generated_ligands = '\n'.join([v['smiles'] for v in molmim_response['generated']])

In [40]:
molmim_response['generated']

[{'smiles': 'Cn1cnc(Cn2c(C3CC3)nnc2N2CCC3(CC2)CNC(=O)O3)n1',
  'score': 0.8442242269672793},
 {'smiles': 'CCC(CC)Cn1c(=O)n(CC)c2nc([C@@H]3C[C@@H]3CF)[nH]c(=O)c21',
  'score': 0.843761038525724},
 {'smiles': 'Cn1cnc(Cn2c(=S)nc(C3CC3)c3c2CCC3)n1',
  'score': 0.8117135601035452},
 {'smiles': 'Cn1cnc(Cn2nnnc2N(CC2CC2)C2CCCCC2)n1',
  'score': 0.8014490677244476},
 {'smiles': 'CCC(CC)(CNC(=O)c1cccc(-c2nc(C3CC3)no2)c1)OC',
  'score': 0.7929591862954889}]

In [41]:
molmim_generated_csv = '../data/molmim_generated_molecules.csv'

# TODO: to be moved into utils.py
def update_dataframe_molmim_generated_molecules(molmim_generated, starting_molecule_name, store_dataframe=True):
    import pandas as pd

    df = pd.DataFrame(molmim_generated)
    # Reset the index and make it a column
    df.reset_index(inplace=True)
    df.rename(columns={'smiles':'generated_smiles',
                    'score':'molmim_qed_score',
                    'index':'generated_compound_index'},
                    inplace=True)
    df['starting_molecule'] = starting_molecule_name

    if store_dataframe:
        df.to_csv(molmim_generated_csv, index=False)

update_dataframe_molmim_generated_molecules(molmim_response['generated'], molecule_name)

### Protein-Ligand Docking with DiffDock

After obtaining the molecules with optimized QED scores, we can predict their binding poses to the receptor target. Here, we apply DiffDock, a state-of-the-art generative model that predicts the 3D structure of a protein-ligand complex, to find out the best (most probable) binding poses. A highlighted feature from DiffDock is that a presumed binding pocket, which usually can be characterized only from experimental 3D structures, is not needed (a.k.a., blind-docking). This feature is very useful for AI folded protein structures, as it is able to locate all regions on the protein surface to be bound by drug molecules, providing ingishts for further downstream investigations.

The optimized DiffDock also provides the batch-docking function, by which we can concatenate multiple molecules into one request of docking, each of them will be also sampled for mulitple poses (i.e., num_poses=10 in this example). In the output, the predicted docking poses for each molecule is sorted by a confidence score that inferenced from a confidence model.

In [25]:
diffdock_response = requests.post(
    f'{DIFFDOCK_HOST}/molecular-docking/diffdock/generate',
    json={
        'protein': folded_protein,
        'ligand': generated_ligands,
        'ligand_file_type': 'txt',
        'num_poses': 10,
        'time_divisions': 20,
        'num_steps': 18,
    }).json()

In [26]:
# overwrite the output directory
def prepare_output_directory(output):
    """
    Prepare the output directory
    output: str, the output directory
    return: None
    """
    # overwrite the output directory
    # delete the output directory if it exists
    if os.path.exists(output):
        shutil.rmtree(output)
    os.makedirs(output)
    
diffdock_output_dir = "../data/diffdock_outputs/"
prepare_output_directory(diffdock_output_dir)

dsmbind_input_dir = "../data/dsmbind_inputs"
prepare_output_directory(dsmbind_input_dir)

In [27]:
for i in range(len(diffdock_response['ligand_positions'])):
    ligand_subfolder_name = molecule_name + "_compound" + str(i)
    ligand_subfolder_in_diffdock = os.path.join(diffdock_output_dir, ligand_subfolder_name)
    prepare_output_directory(ligand_subfolder_in_diffdock)
    with open(f"{diffdock_output_dir}/output.json", "w") as f:
        json.dump(diffdock_response, f)
    
    # save data in DSMBind input format
    ligand_subfolder_in_dsmbind = os.path.join(dsmbind_input_dir, ligand_subfolder_name)
    prepare_output_directory(ligand_subfolder_in_dsmbind)
    # Copy protein file into DSMBind subfolders
    shutil.copy(protein_file_path, ligand_subfolder_in_dsmbind) 

    # save ligand positions
    for j, ligand_geometry in enumerate(diffdock_response["ligand_positions"][i]):
        with open("{}/pose_{}.sdf".format(ligand_subfolder_in_diffdock, j), "w") as f:
            f.write(ligand_geometry)

        if j == 0: # Save the best position for scoring
            with open("{}/pose_{}.sdf".format(ligand_subfolder_in_dsmbind, j), "w") as f:
                f.write(ligand_geometry)
            

In [28]:
%run -i /workspace/bionemo/examples/molecule/dsmbind/infer.py

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
[NeMo W 2024-12-17 19:25:44 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/megatron/core/tensor_parallel/layers.py:257: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
      def forward(
    
[NeMo W 2024-12-17 19:25:44 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/megatron/core/tensor_parallel/layers.py:268: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
      def backward(ctx, grad_output):
    
[NeMo W 2024-12-17 19:25:44 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/megatron/core/tensor_parallel/layers.py:328: FutureWarni

[NeMo I 2024-12-17 19:26:23 megatron_hiddens:110] Registered hidden transform sampled_var_cond_gaussian at bionemo.model.core.hiddens_support.SampledVarGaussianHiddenTransform
[NeMo I 2024-12-17 19:26:23 megatron_hiddens:110] Registered hidden transform interp_var_cond_gaussian at bionemo.model.core.hiddens_support.InterpVarGaussianHiddenTransform


[NeMo W 2024-12-17 19:26:26 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/hydra/_internal/hydra.py:119: UserWarning: Future Hydra versions will no longer change working directory at job runtime by default.
    See https://hydra.cc/docs/1.2/upgrades/1.1_to_1.2/changes_to_job_working_dir/ for more information.
      ret = run_job(
    


[NeMo I 2024-12-17 19:26:26 infer:68] 
    
    ************** Experiment Configuration ***********
[NeMo I 2024-12-17 19:26:26 infer:69] 
    data:
      processed: false
      raw_inference_data_dir: ${oc.env:BIONEMO_HOME}/generative-virtual-screening/data/dsmbind_inputs
      predictions_csv: ${oc.env:BIONEMO_HOME}/generative-virtual-screening/data/dsmbind_predictions.csv
      processed_inference_data_path: ${oc.env:BIONEMO_HOME}/examples/tests/test_data/molecule/dsmbind/inference/processed.pkl
    model:
      hidden_size: 256
      mpn_depth: 3
      num_heads: 4
      dropout: 0.1
      threshold: 10.0
      vocab_size: 38
      aa_size: 21
      max_residue_atoms: 14
    inference:
      num_gpus: 1
      ckpt_path: ${oc.env:BIONEMO_HOME}/models/molecule/dsmbind/dsmbind.pth
    
[NeMo I 2024-12-17 19:26:26 infer:75] ************** Load Trained Model ***********
[NeMo I 2024-12-17 19:26:28 infer:82] # Model parameters: 1019136
[NeMo I 2024-12-17 19:26:29 infer:94] **************

  0%|          | 0/5 [00:00<?, ?it/s][NeMo W 2024-12-17 19:26:30 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'END' at line 2332
      warnings.warn(
    
Failed to patch pandas - unable to change molecule rendering
 20%|██        | 1/5 [00:00<00:00,  9.52it/s][NeMo W 2024-12-17 19:26:30 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'END' at line 2332
      warnings.warn(
    
Failed to patch pandas - unable to change molecule rendering
[NeMo W 2024-12-17 19:26:30 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'END' at line 2332
      warnings.warn(
    
Failed to patch pandas - unable to change molecule rendering
[NeMo W 2024-12-17 19:26:30 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/Bio/PDB/PDBParser.p

[NeMo I 2024-12-17 19:26:30 infer:96] ************** Loading Inference Dataset ***********



100%|██████████| 5/5 [00:00<00:00,  8.89it/s]

[NeMo I 2024-12-17 19:26:30 infer:104] ************** Starting Inference ***********


[NeMo I 2024-12-17 19:26:42 infer:106] Predictions: [-1499583.125, -1607839.375, -1794128.25, -1783053.75, -1524778.625]


In [32]:
df_molmim = pd.read_csv(molmim_generated_csv) 
df_dsmbind = pd.read_csv('../data/dsmbind_predictions.csv')
df_joined = pd.concat([df_molmim, df_dsmbind], axis=1)

df_joined.to_csv('../data/results.csv')